In [8]:
from IPython.core.display import HTML
display(HTML("<style>.container { width:95% !important; }</style>"))   
%load_ext autoreload
%autoreload 1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
from ercollect import molecule as mol
from ercollect.molecule import molecule
from ercollect import rxn_syst
from ercollect.rxn_syst import reaction
import numpy as np
import random
from rdkit.Chem import Draw
from rdkit.Chem import AllChem as Chem
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem.Draw.MolDrawing import MolDrawing, DrawingOptions
from IPython.display import clear_output
from ercollect import SABIO_IO

Author: Andrew Tarzia

Date Created: 15 Nov 2018

Distributed under the terms of the MIT License.

# Notebook to clean up collected SABIO entries
This notebook contains some fixes for problems that occured due to the SABIO database and could not be integrated into the actual code base

# PROBLEM 1:
## 15/11/18
- found that SABIO does not report different UniProt accession numbers for mutants of a sequence.
- as it is not clear which sequence should be used in those cases, I have decided to throw out those reactions

## Step 1:
Determine how many of existing RS have a mutant sequence and set rs.etype attribute

In [24]:
rs_dir = '/home/atarzia/psp/screening_results/new_reactions_sabio_wcharge/'
# rs_dir = '/home/atarzia/psp/screening_results/biomin_search_sabio_wcharge/'

In [26]:
count_mutant = 0
count_total = 0
for rs in rxn_syst.yield_rxn_syst(output_dir=rs_dir):
    EID = rs.DB_ID
    rID = rs.rID
#     print(EID, rID)
    try:
        print(rs.etype)
        continue
    except AttributeError:
        pass
    result = SABIO_IO.get_rxnID_from_eID(eID=EID)
    count_total += 1
    rs.etype = result[4]
    if 'wildtype' not in result[4]:
        print(rs.DB_ID, rs.pkl)
        print(result[4])
        count_mutant += 1
#     break
    rs.save_object(rs_dir+rs.pkl)

1873 sRS-3_11_1_1-SABIO-1873.gpkl
mutant Y128F
1874 sRS-3_11_1_1-SABIO-1874.gpkl
mutant Y128A
1875 sRS-3_11_1_1-SABIO-1875.gpkl
mutant C22A
1876 sRS-3_11_1_1-SABIO-1876.gpkl
mutant C22S
1877 sRS-3_11_1_1-SABIO-1877.gpkl
mutant Y128F/C22S
1878 sRS-3_11_1_1-SABIO-1878.gpkl
mutant H56A
1879 sRS-3_11_1_1-SABIO-1879.gpkl
mutant M49L
44450 sRS-3_11_1_1-SABIO-44450.gpkl
mutant M49L
44451 sRS-3_11_1_1-SABIO-44451.gpkl
mutant G50A
44452 sRS-3_11_1_1-SABIO-44452.gpkl
mutant H56Q
58852 sRS-3_13_1_5-SABIO-58852.gpkl
mutant Phe96Ser
58853 sRS-3_13_1_5-SABIO-58853.gpkl
mutant Phe77Ala
58854 sRS-3_13_1_5-SABIO-58854.gpkl
mutant Gly199Stop
2441 sRS-3_1_1_1-SABIO-2441.gpkl
mutant M211S
2442 sRS-3_1_1_1-SABIO-2442.gpkl
mutant R215L
2443 sRS-3_1_1_1-SABIO-2443.gpkl
mutant M211S/R215L
2444 sRS-3_1_1_1-SABIO-2444.gpkl
mutant M211S/R215L
2445 sRS-3_1_1_1-SABIO-2445.gpkl
mutant M211S/R215L
2446 sRS-3_1_1_1-SABIO-2446.gpkl
mutant M211S/R215L
2447 sRS-3_1_1_1-SABIO-2447.gpkl
mutant M211S/R215L
2448 sRS-3_1_1_1

In [27]:
print(count_mutant, count_total, count_mutant/count_total * 100)

6087 20392 29.849941153393488


## Step 2:
For all RS with 'wildtype' not in rs.etype attribute, set skip_reaction = True and reason

In [28]:
for rs in rxn_syst.yield_rxn_syst(output_dir=rs_dir):
    if 'wildtype' not in rs.etype:
        print(rs.DB_ID, rs.pkl)
        rs.skip_rxn = True
        rs.skip_reason = 'SABIO E-ID is for mutant'
    rs.save_object(rs_dir+rs.pkl)

1873 sRS-3_11_1_1-SABIO-1873.gpkl
1874 sRS-3_11_1_1-SABIO-1874.gpkl
1875 sRS-3_11_1_1-SABIO-1875.gpkl
1876 sRS-3_11_1_1-SABIO-1876.gpkl
1877 sRS-3_11_1_1-SABIO-1877.gpkl
1878 sRS-3_11_1_1-SABIO-1878.gpkl
1879 sRS-3_11_1_1-SABIO-1879.gpkl
44450 sRS-3_11_1_1-SABIO-44450.gpkl
44451 sRS-3_11_1_1-SABIO-44451.gpkl
44452 sRS-3_11_1_1-SABIO-44452.gpkl
58852 sRS-3_13_1_5-SABIO-58852.gpkl
58853 sRS-3_13_1_5-SABIO-58853.gpkl
58854 sRS-3_13_1_5-SABIO-58854.gpkl
2441 sRS-3_1_1_1-SABIO-2441.gpkl
2442 sRS-3_1_1_1-SABIO-2442.gpkl
2443 sRS-3_1_1_1-SABIO-2443.gpkl
2444 sRS-3_1_1_1-SABIO-2444.gpkl
2445 sRS-3_1_1_1-SABIO-2445.gpkl
2446 sRS-3_1_1_1-SABIO-2446.gpkl
2447 sRS-3_1_1_1-SABIO-2447.gpkl
2448 sRS-3_1_1_1-SABIO-2448.gpkl
2449 sRS-3_1_1_1-SABIO-2449.gpkl
2450 sRS-3_1_1_1-SABIO-2450.gpkl
2451 sRS-3_1_1_1-SABIO-2451.gpkl
2452 sRS-3_1_1_1-SABIO-2452.gpkl
2463 sRS-3_1_1_1-SABIO-2463.gpkl
29045 sRS-3_1_1_1-SABIO-29045.gpkl
29046 sRS-3_1_1_1-SABIO-29046.gpkl
29047 sRS-3_1_1_1-SABIO-29047.gpkl
29048 sRS-3_

# PROBLEM 2:
## 15/11/18
- found inaccuracies in the API for modifiers of reactions:
    - in some cases the wrong ion is reported compared to the website or any ion is reported instead of non compared to the website
- as of 23/11/18, we are removing all 'modifier' compounds

## Step 1:
Determine the molecules present in modifier roles

In [19]:
rs_dir = '/home/atarzia/psp/screening_results/new_reactions_sabio_wcharge/'
# rs_dir = '/home/atarzia/psp/screening_results/biomin_search_sabio_wcharge/'

In [20]:
mod_list = []
mod_names = []
for rs in rxn_syst.yield_rxn_syst(output_dir=rs_dir):
    if rs.skip_rxn is True:
        continue
    for m in rs.components:
        if 'modifier' in m.role:  # m.role != 'reactant' and m.role != 'product':
            print(rs.DB_ID, rs.pkl, m.name, m.role)
            mod_list.append((rs.DB_ID, rs.pkl, m.name, m.role))
            mod_names.append(m.name)

41403 sRS-1-SABIO-41403.gpkl Mn2+ modifier-cofactor
41404 sRS-1-SABIO-41404.gpkl Mn2+ modifier-cofactor
41405 sRS-1-SABIO-41405.gpkl Mn2+ modifier-cofactor
44519 sRS-1-SABIO-44519.gpkl 1,8-Dihydroxyanthraquinone modifier-cofactor
44520 sRS-1-SABIO-44520.gpkl 1,8-Dihydroxyanthraquinone modifier-cofactor
44521 sRS-1-SABIO-44521.gpkl 1,8-Dihydroxyanthraquinone modifier-cofactor
44522 sRS-1-SABIO-44522.gpkl 1,8-Dihydroxyanthraquinone modifier-cofactor
44523 sRS-1-SABIO-44523.gpkl 1,8-Dihydroxyanthraquinone modifier-cofactor
44524 sRS-1-SABIO-44524.gpkl 1,8-Dihydroxyanthraquinone modifier-cofactor
46611 sRS-1-SABIO-46611.gpkl FAD modifier-cofactor
58999 sRS-1-SABIO-58999.gpkl Mn2+ modifier-cofactor
36411 sRS-1_13-SABIO-36411.gpkl Fe2+ modifier-cofactor
36412 sRS-1_13-SABIO-36412.gpkl Fe2+ modifier-cofactor
51884 sRS-1_1_1-SABIO-51884.gpkl FAD modifier-cofactor
11020 sRS-1_1_1_1-SABIO-11020.gpkl Zn2+ modifier-cofactor
22371 sRS-1_1_1_1-SABIO-22371.gpkl Zn2+ modifier-cofactor
22372 sRS-1_1_1_

In [21]:
print(len(list(set(mod_names))))
print(list(set(mod_names)))

26
['Glucose 1,6-diphosphate', '1,8-Dihydroxyanthraquinone', 'Pyridoxal phosphate', 'Ca2+', 'Glutathione', 'Pyridoxal 5-phosphate', 'Ni2+', 'Glycerate 2,3-bisphosphate', 'FAD', 'Mn2+', "Pyridoxal 5'-phosphate", 'Co2+', 'FMN', 'D-Glucose 1,6-bisphosphate', 'Zn2+', 'Thiamine diphosphate', 'alpha-D-Glucose 1,6-bisphosphate', 'Fe2+', 'Thiamin diphosphate', 'Glucose 1,6-bisphosphate', 'Reduced glutathione', 'K+', 'Pyruvate', 'MgCl2', 'Mg2+', '2,3-Diphosphoglycerate']


## Step 2:
- Remove components from RS if:
    - role has: 'inhibitor', 'activator' or 'unknown'
    - UPDATE (23/11/18): role has 'modifier' in it.
- save RS  

In [ ]:
for rs in rxn_syst.yield_rxn_syst(output_dir=rs_dir):
    if rs.skip_rxn is True:
        continue
    new_comp = []
    for m in rs.components:
#         if m.role != 'reactant' and m.role != 'product' and 'cofactor' not in m.role:
        if 'modifier' in m.role:
            print(rs.DB_ID, rs.pkl, m.name, m.role)
            continue
        else:
            new_comp.append(m)
    rs.components = new_comp
    rs.save_object(rs_dir+rs.pkl)

56435 sRS-3_2_1_22-SABIO-56435.gpkl Ca2+ modifier-cofactor
56436 sRS-3_2_1_22-SABIO-56436.gpkl Ca2+ modifier-cofactor
34866 sRS-3_2_1_23-SABIO-34866.gpkl Mg2+ modifier-cofactor
34867 sRS-3_2_1_23-SABIO-34867.gpkl Mg2+ modifier-cofactor
34868 sRS-3_2_1_23-SABIO-34868.gpkl Mg2+ modifier-cofactor
34869 sRS-3_2_1_23-SABIO-34869.gpkl Mg2+ modifier-cofactor
34870 sRS-3_2_1_23-SABIO-34870.gpkl Mg2+ modifier-cofactor
34871 sRS-3_2_1_23-SABIO-34871.gpkl Mg2+ modifier-cofactor
34872 sRS-3_2_1_23-SABIO-34872.gpkl Mg2+ modifier-cofactor
34873 sRS-3_2_1_23-SABIO-34873.gpkl Mg2+ modifier-cofactor
34874 sRS-3_2_1_23-SABIO-34874.gpkl Mg2+ modifier-cofactor
35573 sRS-3_2_1_23-SABIO-35573.gpkl Mg2+ modifier-cofactor
35574 sRS-3_2_1_23-SABIO-35574.gpkl Mg2+ modifier-cofactor
35575 sRS-3_2_1_23-SABIO-35575.gpkl Mg2+ modifier-cofactor
35576 sRS-3_2_1_23-SABIO-35576.gpkl Mg2+ modifier-cofactor
35577 sRS-3_2_1_23-SABIO-35577.gpkl Mg2+ modifier-cofactor
35578 sRS-3_2_1_23-SABIO-35578.gpkl Mg2+ modifier-cofact

## Step 3:
- Set all 
    - rs.max_comp_size = None
    - rs.all_fit = None
    - rs.delta_comp
    - rs.delta_sa = None
    - rs.max_XlogP = None
    - rs.min_XlogP = None
    - rs.max_logP = None
    - rs.min_logP = None
    - rs.p_max_comp = None
    - rs.r_max_comp = None
    - rs.p_max_sa = None
    - rs.r_max_sa = None
- rerun rxn_syst.py analysis (external)

In [ ]:
for rs in rxn_syst.yield_rxn_syst(output_dir=rs_dir):
    rs.max_comp_size = None
    rs.all_fit = None
    rs.delta_comp = None
    rs.delta_sa = None
    rs.max_logP = None
    rs.max_XlogP = None
    rs.min_logP = None
    rs.min_XlogP = None
    rs.p_max_comp = None
    rs.p_max_sa = None
    rs.r_max_comp = None
    rs.r_max_sa = None
    rs.save_object(rs_dir+rs.pkl)

## Step 4:
- Delete all reaction files with skip_rxn = True
    - this was done because I could not guarantee that the skips were caused by 'modifiers' or not
- rerun rxn_syst.py collection (external)

In [ ]:
for rs in rxn_syst.yield_rxn_syst(output_dir=rs_dir):
    if rs.skip_rxn is True:
        print(rs.pkl)
        os.system('rm '+rs_dir+rs.pkl)